# Data processing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("data/books.csv")
data.head(3)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN


In [3]:
data.shape

(52478, 25)

Initially, there are over 50'000 records.

## Sanitazing the book formats

In [4]:
counts = data['bookFormat'].nunique()
counts

136

In [5]:
print(data['bookFormat'].unique())

['Hardcover' 'Paperback' 'Mass Market Paperback' 'Kindle Edition'
 'Audiobook' 'ebook' nan 'Board book' 'Boxed Set' 'Leather Bound'
 'Capa dura' 'Trade Paperback' 'Box Set' 'Board Book' 'Nook'
 'Library Binding' 'Capa comum' 'Pasta blanda' 'Audio Cassette'
 'Unknown Binding' 'Audio CD' 'Slipcased Hardcover' 'Broschiert'
 'Paperback ' 'Brochura' 'MP3 CD' 'Audible Audio' 'hardcover' 'cloth'
 'Pasta dura' 'Paperback/Kindle' 'paper' 'Hard Cover' 'Perfect Paperback'
 'Poche' 'Comics' 'Hardcover Slipcased ' 'Unbound' 'Taschenbuch'
 'Paper back' 'Paperback, Kindle, Ebook, Audio' 'CD-ROM'
 'Paperback and Kindle' 'Hardcover im Schuber' 'paperback'
 'Graphic Novels' 'Broché' 'Science Fiction Book Club Omnibus' 'Newsprint'
 'Spiral-bound' 'Mass Market' 'Hardcover Boxed Set'
 'Mass Market Paperback ' 'Hardback' 'Audio' 'Novel' 'Gebundene Ausgabe'
 'softcover' 'گالینگور-وزیری' 'hardbound' 'Hard cover, Soft cover, e-book'
 'Kindle' 'Paperback/Ebook' 'Online Fiction' 'Interactive ebook'
 'Paperback m

In [6]:
# set all names to lower case
data['bookFormat'] = data['bookFormat'].str.lower()
data['bookFormat'] = data['bookFormat'].fillna('unknown binding')

cover_types = data[['bookFormat', 'title']]
cover_types.insert(1, 'count', 1)

n_uniques = cover_types[['bookFormat', 'count']].groupby('bookFormat').sum()
n_uniques.head()

,count
bookFormat,
album,1
audible audio,33
audio,16
audio book,1
audio cassette,36


We can see most of the book formats are either the same or are very marginal. Let's group together into four major groups and put all the other groups into an 'other' category.

In [7]:
n_uniques = n_uniques[n_uniques['count'] > 10] 
n_uniques

,count
bookFormat,
audible audio,33
audio,16
audio cassette,36
audio cd,146
audiobook,107
board book,37
comics,17
ebook,2547
hardcover,12278


In [13]:
format_names = ['hardcover', 'paperback', 'audiobook', 'ebook']
all_cover_names = []
format_categories = [
    ['hardcover', 'slipcased hardcover', 'hardcover slipcased', 'hardcover im schuber', 'hardcover boxed set', 'hardcover, paper dust jacket', 'hardcover chapbook', 'tankobon hardcover'],
    ['mass market paperback', 'paperback', 'trade paperback', 'paperback/ebook', 'paperback, ebook', 'paperback/kindle'],
    ['audible audio', 'audio', 'audio cassette', 'audiobook', 'audio play', 'audio cd'],
    ['ebook', 'kindle edition', 'interactive ebook', 'softcover, free ebook', 'kindle_edition', 'pdf']
]

for i, cover_names in enumerate(format_categories):
    data[format_names[i]] = data.apply(lambda row: row['bookFormat'] in cover_names, axis=1)
    all_cover_names.extend(cover_names)
    
data['other'] = data.apply(lambda row: row['bookFormat'] not in all_cover_names, axis=1)

In [25]:
data[['bookFormat', 'hardcover', 'paperback', 'audiobook', 'ebook', 'other']].sample(10)

,bookFormat,hardcover,paperback,audiobook,ebook,other
35339,paperback,False,True,False,False,False
40817,paperback,False,True,False,False,False
34855,paperback,False,True,False,False,False
6592,paperback,False,True,False,False,False
40378,unknown binding,False,False,False,False,True
27210,paperback,False,True,False,False,False
39333,hardcover,True,False,False,False,False
31965,hardcover,True,False,False,False,False
8285,hardcover,True,False,False,False,False
11933,paperback,False,True,False,False,False
